In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import json
from tqdm import tqdm
plt.style.use('fivethirtyeight')

%matplotlib inline

In [2]:
input_X = np.load("../outputs/10_user_X_Nov_11.npy")
input_y = np.load("../outputs/10_user_y_Nov_11.npy")
num_classes_for_prediction = 10

# put X in the right format
X = np.array([_ for _ in input_X])

# put y in the right format
def ohe(n):
    vec = np.zeros(num_classes_for_prediction)
    vec[n] = 1
    return vec

y = np.array([ohe(n) for n in input_y])
with open('../outputs/Nov11_char_dic.json', 'r') as fp:
    charToIndex = json.load(fp)

# constants
vocab_size = len(charToIndex)
embedding_dims = 300
tweet_size = 140
batch_size = 100
epochs = 10

In [3]:
# Model

# Placeholders for inputs
train_inputs = tf.placeholder(tf.int32, shape=[None, tweet_size], name='train_inputs')
train_labels = tf.placeholder(tf.int32, shape=[None, num_classes_for_prediction], name='train_labels')

# Create the embedding matrix
embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_dims], -1.0, 1.0), name='embeddings')

# Extract the character embeddings for the tweet
embed =  tf.reshape(tf.nn.embedding_lookup(embeddings, train_inputs), [-1, tweet_size, embedding_dims, 1],
                    name='embed')

# The convolutional part
n_gram = 2
num_filters = 32
W_conv = tf.Variable(tf.truncated_normal([n_gram, embedding_dims, 1, num_filters], stddev=0.05), name="W_conv")
b_conv = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b_conv")

conv = tf.nn.conv2d(
        embed,
        W_conv, 
        strides=[1, 1, 1, 1], 
        padding="VALID", 
        name="convolution")

h = tf.nn.tanh(tf.nn.bias_add(conv, b_conv), name="relu")

pooled = tf.nn.max_pool(
        h,
        ksize=[1, h.get_shape()[1].value, 1, 1],
        strides=[1, 1, 1, 1],
        padding='VALID',
        name="max_pool")

# reshape and add FC
h_pool_flat = tf.reshape(pooled, [-1, num_filters], name='flatten')

W_fc = tf.Variable(tf.truncated_normal([num_filters, num_classes_for_prediction], stddev=0.05), name="W_fc")
b_fc = tf.Variable(tf.constant(0.1, shape=[num_classes_for_prediction]), name="b_fc")
fc_output = tf.nn.tanh(tf.matmul(h_pool_flat, W_fc) + b_fc, name="fc")

# cross entropy loss function
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
    _sentinel=None,
    labels=train_labels,
    logits=fc_output,
    dim=-1,
    name='loss'
))

preds = tf.argmax(fc_output, 1, name='predictions')
true_vals = tf.argmax(train_labels, 1, name='true_vals')
acc = tf.reduce_mean(tf.cast(tf.equal(preds, true_vals), tf.float32), name = 'acc')

# SGD optimizer
learning_rate = 0.5
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate, name='optimizer')
train_step = optimizer.minimize(loss, name='train_step')

In [4]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [5]:
# Train data
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)

cv_accuracy_val = []
cv_loss_val = []

split_num = 0
for train_index, test_index in kf.split(X):
    
    # reset the network parameters
    sess.run(init)
    
    print("Split: {}".format(split_num))
    split_num += 1
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # around 2 minutes per epoch
    for e in range(epochs):
        print("Epoch: {}".format(e))
        pairs = zip(np.split(X, indices_or_sections=np.arange(0, len(X), batch_size)), 
                                    np.split(y, indices_or_sections=np.arange(0, len(X), batch_size)))
        train_acc = []
        train_loss = []
        for batch_x, batch_y in tqdm(pairs):
            if len(batch_x)==batch_size:
                feed_dict = {train_inputs: batch_x, train_labels: batch_y}
                _, train_l, train_a = sess.run([train_step, loss, acc], feed_dict=feed_dict)
                train_acc.append(train_a)
                train_loss.append(train_l)
        
        val_acc, val_loss = sess.run([acc, loss], feed_dict={train_inputs: X_test, train_labels: y_test})
        
        print("Training accuracy: {}".format(np.mean(train_acc)))
        print("Training loss: {}".format(np.mean(train_loss)))
        print("Validation accuracy: {}".format(val_acc))
        print("Validation loss: {}".format(val_loss))
    
    cv_loss_val.append(val_loss)
    cv_accuracy_val.append(val_acc)

0it [00:00, ?it/s]

Split: 0
Epoch: 0


901it [02:14,  6.71it/s]
0it [00:00, ?it/s]

Training accuracy: 0.621757447719574
Training loss: 1.3595110177993774
Validation accuracy: 0.7850789427757263
Validation loss: 1.114120602607727
Epoch: 1


901it [02:00,  7.48it/s]
0it [00:00, ?it/s]

...

Training accuracy: 0.8912346959114075
Training loss: 0.9487499594688416
Validation accuracy: 0.8835269808769226
Validation loss: 0.9609959125518799
Epoch: 8


901it [01:56,  7.71it/s]
0it [00:00, ?it/s]

Training accuracy: 0.8933815360069275
Training loss: 0.9458463788032532
Validation accuracy: 0.8795797228813171
Validation loss: 0.968978762626648
Epoch: 9


901it [01:57,  7.65it/s]


Training accuracy: 0.8939710259437561
Training loss: 0.9431775808334351
Validation accuracy: 0.8815255165100098
Validation loss: 0.9644759893417358


In [11]:
print("CV Accuracy: {}".format(np.mean(cv_loss_val)))
print("CV Loss: {}".format(np.mean(cv_accuracy_val)))

CV Accuracy: 0.9641767740249634
CV Loss: 0.8712984919548035


In [16]:
# Saved Model, DO NOT RERUN
# save_dir = "../models/"
# model_name = "10_class_1_layerconv_89%"
# saver = tf.train.Saver()
# saver.save(sess, save_dir+model_name)

In [17]:
# restore model
saver = tf.train.Saver()
saver.restore(sess, "../models/10_class_1_layerconv_89%")

INFO:tensorflow:Restoring parameters from ../models/10_class_1_layerconv_89%


In [18]:
sess.run([acc, loss], feed_dict={train_inputs: X[:1000], train_labels: y[:1000]})

[0.89999998, 0.9561885]

In [19]:
# Extract the embeddings and save them

[char_embeddings] = sess.run([embeddings])

# save these to a file
np.save("../outputs/10_user_Nov11_embeddings.npy", char_embeddings)